<a href="https://colab.research.google.com/github/juniordamazeno/IA_POS/blob/main/DATASET_Canc%C3%AAr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch torchvision scikit-learn matplotlib

import torch
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os
import numpy as np

# Define transformações para o modelo ViT
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

# Carregar imagens
def load_image(path):
    image = Image.open(path).convert("RGB")
    return transform(image)

# Carregar dados
def load_dataset(folder, label):
    images = []
    labels = []
    for file in os.listdir(folder):
        if file.endswith('.jpg') or file.endswith('.png'):
            try:
                img = load_image(os.path.join(folder, file))
                images.append(img)
                labels.append(label)
            except:
                continue
    return images, labels

# Substitua pelos caminhos corretos
normal_imgs, normal_lbls = load_dataset("/content/cancer", 0)
cancer_imgs, cancer_lbls = load_dataset("/content/normal", 1)

X = normal_imgs + cancer_imgs
y = normal_lbls + cancer_lbls

# Dividir dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Converter para tensores
X_train_tensor = torch.stack(X_train)
X_test_tensor = torch.stack(X_test)
y_train_tensor = torch.tensor(y_train)
y_test_tensor = torch.tensor(y_test)

# Carregar modelo ViT pré-treinado
model = models.vit_b_16(pretrained=True)
model.heads.head = torch.nn.Linear(model.heads.head.in_features, 2)  # 2 classes

# Treinamento
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

model.train()
for epoch in range(3):  # poucas épocas por exemplo
    inputs = X_train_tensor.to(device)
    targets = y_train_tensor.to(device)
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# Avaliação
model.eval()
with torch.no_grad():
    preds = model(X_test_tensor.to(device))
    y_pred = torch.argmax(preds, dim=1).cpu().numpy()
    print(classification_report(y_test, y_pred))


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth


100%|██████████| 330M/330M [00:04<00:00, 76.5MB/s]


Epoch 1, Loss: 0.7071
Epoch 2, Loss: 0.1339
Epoch 3, Loss: 0.2051
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

